Based on the information provided in the "Optym-Take-Home-Exam.pdf" document, we can develop a Mixed Integer Linear Programming (MILP) model for the Aircraft Route Optimization Problem. The model will include complete notations for input parameters, decision variables, constraints, and the objective function.

# Input Parameters:


1. <b>D</b>: Set of demands (customer flying requests).
2. <b>A</b>: Set of aircraft.
3. <b>L</b>: Set of flight legs.
4. <b>T</b>: Time periods for planning (e.g., days).
5. <b>C<sub>l</sub></b>: Operating cost for each flight leg <i>l ∈ L</i>.
6. <b>R<sub>d</sub></b>: Revenue from fulfilling demand <i>d ∈ D</i>.
7. <b>M</b>: Large positive number (for modeling purposes).
8. <b>S<sub>la</sub></b>: Starting location of aircraft <i>a ∈ A</i> for leg <i>l ∈ L</i>.
9. <b>E<sub>la</sub></b>: Ending location of aircraft <i>a ∈ A</i> for leg <i>l ∈ L</i>.
10. <b>HOS<sub>a</sub></b>: Maximum hours of service for aircraft <i>a ∈ A</i>.
11. <b>Rest<sub>a</sub></b>: Mandatory rest period for aircraft <i>a ∈ A</i> after <i>HOS<sub>a</sub></i> is reached.




# Decision Variables:

1.	x_lat: Binary variable, 1 if aircraft a flies leg l at time t, 0 otherwise.
2.	y_dat: Binary variable, 1 if demand d is fulfilled at time t, 0 otherwise.
3.	z_la: Binary variable, 1 if leg l is flown empty by aircraft a, 0 otherwise.


# Objective Function:

Maximize total profit, which is the revenue from fulfilling demands minus the operating costs:


$$
\text{Maximize } Z = \sum_d \sum_t R_d \cdot y_{dat} - \sum_l \sum_a \sum_t C_l \cdot x_{lat}
$$


## import libraries

In [696]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary, LpStatus
import pandas as pd
import pickle

## load data

In [697]:
aircraft_data = pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Aircraft.csv")
airports_data = pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Airports.csv", encoding='ISO-8859-1')
demands_file =  pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Demands.csv")
distance_time_file =  pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Distance and Flying Time.csv")

# scenario 1D: Focus on a single day: 24th July

In [698]:
# specific_date = pd.Timestamp('2022-07-24')
# demands_file['ScheduledDepDatetime'] = pd.to_datetime(demands_file['ScheduledDepDatetime'])
# demands_file = demands_file[demands_file['ScheduledDepDatetime'].dt.date == specific_date.date()]

# scenario 2D: Focus on a single day: 24th and 25th July

In [699]:
specific_date = [pd.Timestamp('2022-07-24'), pd.Timestamp('2022-07-25')]
demands_file['ScheduledDepDatetime'] = pd.to_datetime(demands_file['ScheduledDepDatetime'])
demands_file = demands_file[demands_file['ScheduledDepDatetime'].dt.date.isin([date.date() for date in specific_date])]


## preprocess

In [700]:
aircraft_set = aircraft_data['AircraftID'].unique()
demands_set = demands_file['DemandID'].unique()
airports_set = airports_data['Airport Name'].unique() 
L = list(set(zip(distance_time_file['DepAirport'], distance_time_file['ArrAirport'])))
time_periods = [date.date() for date in specific_date]

input size is so big so here is how to reduce and filter the ac set and L

In [701]:
# Filtering aircraft_set based on initial location
preferred_locations = {'Location1', 'Location2'}  # Replace with actual locations of interest
filtered_aircraft_set = [a for a in aircraft_set if aircraft_data[aircraft_data['AircraftID'] == a]['InitialLocation'].iloc[0] in preferred_locations]

# Filtering L based on specific airport pairs
relevant_airport_pairs = set(zip(distance_time_file['DepAirport'], distance_time_file['ArrAirport']))  # You can further filter this set as needed
filtered_L = [(dep, arr) for (dep, arr) in L if (dep, arr) in relevant_airport_pairs]


## parameters

In [702]:
fixed_cost_per_leg = 10000
# Operating cost for each flight leg 
operating_costs = {row['DepAirport'] + row['ArrAirport']: row['FlyingTime']
                   for _, row in distance_time_file.iterrows()}
#
# Revenue from fulfilling demand 
revenue_per_demand = {row['DemandID']: row['PaxCount'] * 100  # Arbitrary revenue per passenger
                      for _, row in demands_file.iterrows()}

In [703]:
# Create a dictionary mapping each flight leg to its estimated block time (if demand exists)
demand_block_time = {(row['DepAirport'], row['ArrAirport']): row['EstimatedBlocktime']
                     for _, row in demands_file.iterrows() if not pd.isna(row['EstimatedBlocktime'])}

# Create a dictionary mapping each flight leg to its flying time
flying_time = {(row['DepAirport'], row['ArrAirport']): row['FlyingTime']
               for _, row in distance_time_file.iterrows()}

# Initialize operating_costs dictionary
operating_costs = {}


In [704]:
# Update travel time values for each flight leg
for l in filtered_L:
    dep_airport, arr_airport = l

    # Check if there is a demand associated with this flight leg
    if l in demand_block_time:
        # Flight leg has an associated demand, use pre-calculated estimated block time
        travel_time = demand_block_time[l]
    else:
        # Flight leg is empty, use pre-calculated travel time values
        travel_time = flying_time.get(l, 0)  # Default to 0 if not found

        # Check if the travel time exceeds the maximum flight range
        if travel_time > 4.5:
            # Flight leg requires refueling, add one hour to the travel time
            travel_time += 1

    # Update the travel time for this flight leg in the operating_costs dictionary
    operating_costs[l] = travel_time


## Model initialization

In [705]:
model = LpProblem("Aircraft_Route_Optimization", LpMaximize)

## Decision Variables

In [706]:
y_vars = LpVariable.dicts("y", [(d, a, t) for d in demands_set for a in filtered_aircraft_set for t in time_periods], cat=LpBinary)

In [707]:
x_vars = LpVariable.dicts("x", 
                          [(a, l, t) for a in filtered_aircraft_set for l in filtered_L for t in time_periods], 
                          cat=LpBinary)

In [708]:
z_vars = LpVariable.dicts("z", 
                          [(a, l) for a in filtered_aircraft_set for l in filtered_L], 
                          cat=LpBinary)

In [709]:
empty_leg_vars = LpVariable.dicts("EmptyLeg", 
                                  [(a, l, t) for a in filtered_aircraft_set for l in filtered_L for t in time_periods], 
                                  cat=LpBinary)

In [710]:
# x_vars = LpVariable.dicts("x", [(a, l, t) for a in aircraft_set for l in L for t in time_periods], cat=LpBinary)
# y_vars = LpVariable.dicts("y", [(d, t) for d in demands_set for t in time_periods], cat=LpBinary)
# z_vars = LpVariable.dicts("z", [(a, l) for a in aircraft_set for l in L], cat=LpBinary)

# Constraints

1. -	Demand Fulfillment: Each demand must be fulfilled at least once in the planning period.


$$\sum_{t} y_{dt} \geq 1 \quad \text{for all } d$$


In [711]:
# for d in demands_set:
#     model += lpSum(y_vars[d, a, t] for a in filtered_aircraft_set for t in time_periods) >= 1, f"Demand_Fulfillment_{d}"



2. -	Aircraft Route Continuity: Ensures that for each aircraft, the number of arrivals at an airport equals the number of departures.

$$\sum_{l, S_{la} = k} X_{lat} = \sum_{l, E_{la} = k} X_{lat} \quad \text{for all } a, t, \text{ and } k \text{ in airports}$$


In [712]:
# for a in filtered_aircraft_set:
#     for k in airports_set:  
#         for t in time_periods:
#             arrivals = lpSum(x_vars[a, (dep, arr), t] for (dep, arr) in filtered_L if arr == k)
#             departures = lpSum(x_vars[a, (dep, arr), t] for (dep, arr) in filtered_L if dep == k)
#             model += (arrivals == departures), f"Aircraft_Route_Continuity_{a}_{k}_{t}"


3-	Aircraft Utilization Limit: Each aircraft cannot exceed its maximum hours of service.

$$\sum_{l} \sum_{t} x_{lat} \leq \text{HOS}_a \quad \text{for all } a$$


In [713]:
# for a in filtered_aircraft_set:
#     for t in time_periods:
#         total_hours_of_operation = lpSum(x_vars[a, l, t] for l in filtered_L)
#         # Allow for flexibility: Aircraft can be left unused or used < 12.5 hours
#         model += (total_hours_of_operation <= 12.5), f"Utilization_Limit_{a}_{t}"


4. 	Mandatory Rest: After reaching the maximum HOS, the aircraft must observe a mandatory rest period.

$$
\begin{aligned}
&\text{If } \quad l \in L, \quad \sum x_{lat} = \text{HOS}_a, \text{ then} \\
&\quad l \in L, \quad \sum x_{la(t+1)} = 0
\end{aligned}
$$


Given the complexity of directly implementing this as a linear constraint, a practical approach might be to set a utilization limit slightly less than the maximum HOS for each day, thereby implicitly allowing for rest time. This approach simplifies the model while achieving the intended outcome of ensuring rest periods.

In [714]:
# for a in filtered_aircraft_set:
#     for t in range(len(time_periods) - 1): 
#         current_time = time_periods[t]
#         next_time = time_periods[t + 1]
#         time_difference = (next_time - current_time).total_seconds() / 3600

#         if time_difference >= 12.5:
#             model += (time_difference >= 22.5), f"Mandatory_Rest_{a}_{current_time}"



5.	Flight Leg Assignment: A flight leg can only be assigned if it is either loaded or flown empty.

$$x_{lat} \leq M \cdot z_{la} \quad \text{for all } l, a, \text{ and } t$$


In [715]:
# M = 10000  
# for a in filtered_aircraft_set:
#     for l in filtered_L:
#         for t in time_periods:
#             model += x_vars[a, l, t] <= M * z_vars[a, l], f"Flight_Leg_Assignment_{a}_{l}_{t}"


In [716]:
# for l in filtered_L:
#     # Ensure that the sum of assignment variables for this flight leg is at most 1
#     model += lpSum(x_vars[a, l, t] for a in filtered_aircraft_set for t in time_periods) <= 1, f"Assignment_{l}"

In [717]:
# for a in filtered_aircraft_set:
#     for t in time_periods:
#         max_capacity = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'MaxPax'].iloc[0]
#         assigned_passengers = lpSum(revenue_per_demand[d] * y_vars[d, t] for d in demands_set if (d, t) in y_vars)
        
#         # Add the constraint that ensures passenger capacity is not exceeded
#         model += assigned_passengers <= max_capacity, f"Passenger_Capacity_{a}_{t}"


In [718]:
# # Add a constraint to ensure turn-around time is honored for each aircraft
# for a in filtered_aircraft_set:
#     for t in time_periods[:-1]: 
#         turn_around_time = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'TurnAroundTime'].iloc[0]
#         available_time = lpSum(x_vars[a, l, t_prev] for l in filtered_L for t_prev in time_periods if t_prev < t) + turn_around_time
        
#         # Ensure that the aircraft becomes available for the next flight after the turn-around time
#         model += available_time <= lpSum(x_vars[a, l, t] for l in filtered_L)


In [719]:
# # Define the constraint for each aircraft
# for a in filtered_aircraft_set:
#     model += lpSum(x_vars[a, l, t] for l in filtered_L for t in time_periods) <= 1, f"Single_Route_Constraint_{a}"


In [720]:
# # Ensure that each demand is assigned to at most one aircraft
# for d in demands_set:
#     model += lpSum(y_vars[d, a, t] for a in filtered_aircraft_set for t in time_periods) <= 1, f"Single_Aircraft_Assignment_{d}"

In [721]:
# # Ensure that all demands are satisfied
# for d in demands_set:
#     model += lpSum(y_vars[d, a, t] for a in filtered_aircraft_set for t in time_periods) >= 1, f"Satisfy_Demand_{d}"


In [722]:
# # Ensure that aircraft departure time aligns with scheduled departure datetime of demands
# for d in demands_set:
#     for a in filtered_aircraft_set:
#         for t in time_periods:
#             # Use a binary variable to indicate if the demand is served by the aircraft at time t
#             model += x_vars[a, d, t] <= y_vars[d, a, t], f"Departure_Sync_{a}_{d}_{t}"


In [723]:
# # Ensure that the first flight leg of each aircraft starts from its initial available location
# for a in filtered_aircraft_set:
#     initial_location = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'InitialLocation'].iloc[0]
#     # For each potential destination from the initial location
#     for arr_airport in airports_set:
#         if (initial_location, arr_airport) in filtered_L:
#             # Ensure that if the aircraft flies at the first time period, it departs from its initial location
#             model += x_vars[a, (initial_location, arr_airport), time_periods[0]] <= z_vars[a, (initial_location, arr_airport)], f"Initial_Departure_{a}_{initial_location}_{arr_airport}"



In [724]:
# # Constraint to ensure that each aircraft starts from its initial location
# for a in filtered_aircraft_set:
#     initial_location = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'InitialLoca'].iloc[0]
#     model += lpSum(x_vars[a, (initial_location, dest), time_periods[0]] 
#                    for dest in airports_set if (initial_location, dest) in filtered_L) == 1, f"Start_From_Initial_Location_{a}"


In [725]:
# # Constraint to ensure that the total scheduled hours do not exceed the Initial HOS for each aircraft
# for a in filtered_aircraft_set:
#     initial_hos = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'InitialHOS'].iloc[0]
#     # Assuming 'operating_costs' is a dictionary with keys as (dep, arr) tuples and values as hours
#     model += lpSum(operating_costs[(dep, arr)] * x_vars[a, (dep, arr), t] 
#                    for (dep, arr) in filtered_L for t in time_periods) <= initial_hos, f"Max_HOS_{a}"


In [726]:
# # Add constraints to define when a leg is empty
# for a in filtered_aircraft_set:
#     for l in filtered_L:
#         for t in time_periods:
#             # An empty leg is one that is flown (x_vars) but not servicing a demand (y_vars)
#             # Assuming 'd' represents demand legs, modify as necessary for your code
#             model += empty_leg_vars[a, l, t] >= x_vars[a, l, t] - lpSum(y_vars[d, a, t] for d in demands_set if d == l), f"Empty_Leg_Definition_{a}_{l}_{t}"

## objective function


In [727]:
# Objective Function Components

# Revenue: Sum of revenue for fulfilling each demand
revenue = lpSum(revenue_per_demand[d] * y_vars[d, a, t] for d in demands_set for a in filtered_aircraft_set for t in time_periods)

# Operational Costs: Sum of operating costs for all flight legs
operational_cost = lpSum(operating_costs[l] * x_vars[a, l, t] for a in filtered_aircraft_set for l in filtered_L for t in time_periods)

# Empty Leg Costs: Assuming you have a penalty cost for each empty leg
# Replace 'empty_leg_penalty' with the actual penalty cost per empty leg
empty_leg_penalty = 100000  # Example penalty cost, adjust as needed
empty_leg_cost = lpSum(empty_leg_penalty * empty_leg_vars[a, l, t] for a in filtered_aircraft_set for l in filtered_L for t in time_periods)

# Defining the Objective Function
# Objective: Maximize profit (revenue - operational cost - empty leg cost)
model += revenue - operational_cost - empty_leg_cost, "Total_Profit"


## solve the model

Focus of optimization search on maximization of total profit

In [728]:
# Solving the Model
model.solve()

if LpStatus[model.status] == 'Optimal':
    print("Optimal Solution Found!")
    # Print the values of decision variables
    for var in model.variables():
        if var.varValue is not None and var.varValue > 0:
            print(var.name, "=", var.varValue)
else:
    print("No optimal solution found. Status:", LpStatus[model.status])

# model.writeLP("model.lp")

Optimal Solution Found!


output csv